In [ ]:
# # 1. Nettoyer TOUTES les versions d'OpenCV
# !pip uninstall -y opencv-python opencv-python-headless

# # 2. Réinstaller proprement
# # Nous installons 'headless' (sans interface graphique, idéal pour les notebooks)
# # et nous forçons la réinstallation d'ultralytics pour qu'il s'adapte.
# !pip install -q --no-cache-dir opencv-python-headless ultralytics --force-reinstall

In [ ]:
# !pip uninstall -y numpy opencv-python opencv-python-headless 

# !pip install -q --no-cache-dir ultralytics

In [ ]:
# # Force la réinstallation de ultralytics et met à jour Pillow
# !pip install -q --no-cache-dir ultralytics Pillow --force-reinstall --upgrade

In [ ]:
# %%writefile usure.yaml
# # Chemin absolu vers le dataset que vous avez uploadé
# path: /kaggle/input/data-tu-yolo/data_tu_yolo

# # Chemins relatifs à 'path'
# train: images/train
# val: images/val
# test: images/test

# # Noms des classes
# names:
#   0: 'temoin:0'
#   1: 'temoin:25'
#   2: 'temoin:50'
#   3: 'temoin:75'
#   4: 'temoin:80'
#   5: 'temoin:90'
#   6: 'temoin:95'
#   7: 'temoin:100'

In [1]:
from ultralytics import YOLO

# 1. Charger un modèle
model = YOLO('yolov8n.pt') # 'n' (nano) est le plus rapide

# 2. Entraîner
results = model.train(
    data='usure.yaml',  # Le fichier qu'on vient de créer
    epochs=50,          # Augmentez les époques
    imgsz=300,          # Taille d'image
    patience=10,        # Arrêt anticipé
    project='usure_yolo', # Dossier de sauvegarde
    name='run1'
)

Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.9.1+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=usure.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=300, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True

In [2]:
# 'model' est déjà le meilleur modèle, pas besoin de recharger.
# On évalue sur le set 'test' défini dans le YAML
print("=== Évaluation sur le Test Set ===")
metrics = model.val(split='test')

print("\n--- Métriques principales ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"   mAP50: {metrics.box.map50:.4f}")
print(f"   mAP75: {metrics.box.map75:.4f}")

=== Évaluation sur le Test Set ===
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.9.1+cu128 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 24.2±11.9 MB/s, size: 87.8 KB)
val: Scanning /kaggle/input/data-tu-yolo/data_tu_yolo/labels/test... 141 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 141/141 293.1it/s 0.5s0.1s
WARNING ⚠️ val: Cache directory /kaggle/input/data-tu-yolo/data_tu_yolo/labels is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 6.1it/s 1.5s0.1s
                   all        141        141       0.97      0.942      0.961      0.767
              temoin:0          7          7       0.98      0.857      0.873      0.768
             temoin:25         18         18      0.994      0.944      0.992      0.725
             temoin:50         16         16       0.88     

In [8]:
# Cellule 6 : Prédiction sur une image (VOTRE CODE CORRIGÉ)
import os
from PIL import Image

# 1. Définir le chemin de l'image source
test_img_dir = '/kaggle/input/data-tu-yolo/data_tu_yolo/images/test/'
test_image_name = os.listdir(test_img_dir)[20]
test_image_path = os.path.join(test_img_dir, test_image_name)

print(f"Prédiction sur : {test_image_path}")

# 2. Faire la prédiction
pred_results = model.predict(test_image_path, conf=0.2)

# --- CORRECTION ---
# 3. S'assurer que le dossier de sortie existe
output_dir = '/kaggle/working/outputs'
os.makedirs(output_dir, exist_ok=True) # <--- CETTE LIGNE EST AJOUTÉE
output_img_path = os.path.join(output_dir, f'prediction_{test_image_name}')
# --- FIN CORRECTION ---

# 4. Sauvegarder l'image
res_plotted = pred_results[0].plot()
Image.fromarray(res_plotted[..., ::-1]).save(output_img_path)

print(f"Prédiction sauvegardée : {output_img_path}")

Prédiction sur : /kaggle/input/data-tu-yolo/data_tu_yolo/images/test/20190613_154644.jpg

image 1/1 /kaggle/input/data-tu-yolo/data_tu_yolo/images/test/20190613_154644.jpg: 320x256 1 temoin:50, 9.1ms
Speed: 1.4ms preprocess, 9.1ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 256)
Prédiction sauvegardée : /kaggle/working/outputs/prediction_20190613_154644.jpg


In [6]:
/kaggle/input/test-yolo-gemini/image_test.png

# Cellule 6 : Prédiction sur une image (VOTRE CODE CORRIGÉ)
import os
from PIL import Image


test_image_path = 

print(f"Prédiction sur : {test_image_path}")

# 2. Faire la prédiction
pred_results = model.predict(test_image_path, conf=0.2)

# --- CORRECTION ---
# 3. S'assurer que le dossier de sortie existe
output_dir = '/kaggle/working/outputs'
os.makedirs(output_dir, exist_ok=True) # <--- CETTE LIGNE EST AJOUTÉE
output_img_path = os.path.join(output_dir, f'prediction_{test_image_name}')
# --- FIN CORRECTION ---

# 4. Sauvegarder l'image
res_plotted = pred_results[0].plot()
Image.fromarray(res_plotted[..., ::-1]).save(output_img_path)

print(f"Prédiction sauvegardée : {output_img_path}")

Prédiction sur : /kaggle/input/data-tu-yolo/images/test/20190705_141224.jpg



FileNotFoundError: /kaggle/input/data-tu-yolo/images/test/20190705_141224.jpg does not exist